# Reinforcement Learning: Min Distance RL with CARLA CL (Larger Model) 13/01/22

At the recommednation of one of the authors of the paper I will try to adapt the following [tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) to work acheive the same results as in the paper.

Notes: Pygame only runs on python 3.7, pytorch must be install direclty into the env

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

from torch.utils.tensorboard import SummaryWriter

import gym
import gym_carla

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# parameters for the gym_carla environment
params = {
    'number_of_vehicles': 0,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 3,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'use_fixed':True,
    'weather':'HardRainSunset',
    'obs_size':128
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=10.000000, cloudiness=10.000000, precipitation=0.000000, precipitation_deposits=10.000000, wind_intensity=30.000000, sun_azimuth_angle=150.000000, sun_altitude_angle=60.000000, fog_density=40.000000, fog_distance=60.000000, fog_falloff=2.000000, wetness=30.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)
372


In [3]:

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    
    def __len__(self):
        return len(self.memory)


In [4]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6 = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(64,512, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(512,256, kernel_size =4, stride=2, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(256,128, kernel_size =4, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(128,64, kernel_size =4, stride=2, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=2, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        self.conv12 = torch.nn.ConvTranspose2d(32,23, kernel_size =4, stride=2,padding=1)
        
            
    def encode(self, x):
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        return self.conv6(x)
    
    def decode(self, x):
        x = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn7(self.conv8(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn8(self.conv9(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn9(self.conv10(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn10(self.conv11(x)),negative_slope=0.02)
        return F.log_softmax(self.conv12(x),dim=1)
    
    def forward(self, x):
        x = x.to(device, dtype=torch.float32)
        latent = self.encode(x)
        out = self.decode(latent)
        return out, latent

In [5]:
class Full_DQN(nn.Module):

    def __init__(self, outputs,input_size):
        super(Full_DQN, self).__init__()
        
        self.lin1 = nn.Linear(input_size+9,80)
        self.lin2 = nn.Linear(80,50)
        self.lin3 = nn.Linear(50,25)
        self.lin4 = nn.Linear(25,15)
        self.lin5 = nn.Linear(15,8)
        self.lin6 = nn.Linear(8,3)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.relu(self.lin5(x))
        x = self.lin6(x)
        
        return x.view(x.size(0), -1)

In [6]:
BATCH_SIZE = 512
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200000
TARGET_UPDATE = 256

writer = SummaryWriter()

n_actions = env.action_space.n

policy_net = Full_DQN(n_actions,64).to(device)
#policy_net.load_state_dict(torch.load('./model_params_CL/model_10.final'))

target_net = Full_DQN(n_actions,64).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

model = PerceptionNet()
model.to(device)
model.load_state_dict(torch.load('./AE_params/model_3.best'))
model.eval()
for param in model.parameters():
    param.requires_grad = False

# Model <= 7
optimizer = optim.RMSprop(policy_net.parameters(),lr=0.005)

#Model 8 & from 2022 
#optimizer = optim.Adam(policy_net.parameters(),lr=0.005)


memory = ReplayMemory(7500)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            #arg max select the idex of the largest value and view changes shape from (1,) to (1,1)
            #try test net
            return policy_net(state.float()).argmax().view(1,1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    #reshape state_batch for nn
  
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    
    # selects column of output that was selceted 
    state_action_values = policy_net(torch.reshape(state_batch,(BATCH_SIZE,1,9)).float()).gather(1,action_batch)
    
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(torch.reshape(non_final_next_states,(list(non_final_next_states.shape)[0]//9,1,9)).float()).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp(-1, 1)
    optimizer.step()
    
    return loss

In [13]:
num_episodes = 600
use_fixed_idx = 0
env.use_fixed = 'M'
levels = ['E','M','H']
next_lvl = 300
min_overall_loss = 1000
for i_episode in range(num_episodes):
    rewards = []
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state)

    
    new_obs = torch.tensor(obs['camera'])
    new_obs = new_obs.permute(2,0,1).reshape(1,3,128,128)
    
    _,latent_space = model(new_obs)
    state = torch.cat((state,latent_space.cpu()),-1)
    latent_space = latent_space.reshape(1,64)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an action
        action = select_action(state)
        obs, reward, done, info  = env.step(action.item())
        rewards.append(reward)
        reward = torch.tensor([reward], device=device)

        if not done:
            
            #pos gets a distanc d and array w which has to be seperated out in below line
            pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
            ang = np.asarray(info['angular_vel'])
            acc = np.asarray(info['acceleration'])
            steer = np.asarray(info['steer'])
            next_state = np.concatenate((pos, ang, acc, steer), axis=None)
            
            new_obs = torch.tensor(obs['camera'])
            new_obs = new_obs.permute(2,0,1).reshape(1,3,128,128)
            _,latent_space = model(new_obs)
            latent_space = latent_space.reshape(1,64)
            next_state = torch.cat((torch.tensor(next_state),latent_space),1)
        else:
            next_state = None

        memory.push(state, action, next_state, reward)
        
        state = next_state

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        
        #if loss is better than previous minimum save mode
        if loss is not None and loss < episode_loss:
            episode_loss = loss
            writer.add_scalar("Loss/train", loss, i_episode)
        
        if done:
            episode_durations.append(t + 1)
            break
            
    # Update the target network
    if steps_done % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    len_episode = t+1
    writer.add_scalar("Lenght/Epoch", len_episode, i_episode)
    
    if i_episode > 0 and i_episode % next_lvl == 0:
        #saving model after each level
        torch.save(target_net.state_dict(), './model_params_CL/Full_model_1'+levels[use_fixed_idx]+'.final')
        use_fixed_idx += 1
        env.use_fixed = levels[use_fixed_idx]
    
    #save model if better than previous episode
    if episode_loss < min_overall_loss:
        min_overall_loss = episode_loss
        torch.save(target_net.state_dict(), './model_params_CL/Full_model_1.best')
    

print('Complete')
torch.save(target_net.state_dict(), './model_params_CL/Full_model_1.final')

writer.flush()
writer.close()
env.close()

RuntimeError: Tensors must have same number of dimensions: got 1 and 4

### Visualise Models

In [6]:
target_net.eval()

DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [23]:
torch.save(target_net.state_dict(), './model_params_CL/model_10.final')

In [7]:
vis_net = DQN(n_actions).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_10.final'))
vis_net.eval()



DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [8]:
num_episodes = 8
env.use_fixed = 'H'
env.route_idx = 1
levels = ['E','M','H']
next_lvl = 4
min_overall_loss = 1000
for i_episode in range(num_episodes):
    rewards = 0
    # Initialize the environment and state
    obs = env.reset()
    #ego_dir retirves the distance and angle from vehicle to nearest waypoint
    ego_location = env.ego.get_location()
    ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
    #pos gets a distanc d and array w which has to be seperated out in below line
    ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
    state = np.concatenate((ego_pos,np.zeros(6)))
    state = torch.tensor(state)
    # Resize, and add a batch dimension (BCHW)
    loss = episode_loss = 1000
    for t in count():
        # Select and perform an actionii#
        with torch.no_grad():
            print(state.float())
            action = vis_net(state.float()).argmax().view(1,1)
            print(vis_net(state.float()))
            print(action)
        _ , reward, done, info  = env.step(action.item())
        rewards += reward
        
        #pos gets a distanc d and array w which has to be seperated out in below line
        pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
        ang = np.asarray(info['angular_vel'])
        acc = np.asarray(info['acceleration'])
        steer = np.asarray(info['steer'])
        next_state = np.concatenate((pos, ang, acc, steer), axis=None)
        #update state
        state = torch.tensor(next_state)

        if done:
            print('########')
            episode_durations.append(t + 1)
            break


tensor([0.0035, 1.0000, 0.0057, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
tensor([[4.1582],
        [4.1682],
        [4.1680]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03, -3.5187e-06, -6.6744e-06,
        -5.2950e-09, -5.2512e-09, -8.2394e-07, -0.0000e+00])
tensor([[4.1582],
        [4.1682],
        [4.1680]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([3.4923e-03, 9.9998e-01, 5.6919e-03, 4.7972e-05, 6.7963e-06, 7.4775e-10,
        4.1868e-07, 9.2484e-07, -0.0000e+00])
tensor([[4.1582],
        [4.1682],
        [4.1680]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03,  5.1451e-06, -6.4934e-06,
        -3.9681e-09, -4.1333e-07, -1.5309e-06, -0.0000e+00])
tensor([[4.1582],
        [4.1682],
        [4.1680]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 3.4923e-03,  9.9998e-01,  5.6919e-03, -4.7999e-05,  6.2347e-06,
         5.5763e-09,  4.2232e-07, -4.125

tensor([ 3.0257e-01,  9.9998e-01,  5.6919e-03, -4.2522e+00,  1.7389e-01,
        -1.4912e+01,  3.5015e+00, -5.0069e+00, -3.0000e-01])
tensor([[3.9806],
        [3.9568],
        [3.9138]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 0.3694,  1.0000,  0.0057,  4.9091,  0.5957,  5.5808, -0.4915,  4.1476,
         0.3000])
tensor([[3.8478],
        [3.7965],
        [3.7196]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 3.8911e-01,  9.9998e-01,  5.6919e-03,  2.3591e+00,  1.5934e+00,
         1.8356e+01, -3.0944e+00,  4.6439e+00,  3.0000e-01])
tensor([[3.7836],
        [3.7190],
        [3.6258]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 3.8646e-01,  9.9998e-01,  5.6919e-03, -1.8817e-01,  1.3716e+00,
         1.7777e+01, -3.0467e+00,  1.4850e+00,  3.0000e-01])
tensor([[3.9129],
        [3.8751],
        [3.8149]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 3.7469e-01,  9.9998e-01,  5.6919e-03, -5.5994e-01,  8.4908e-01,
         1.6256

tensor([ 2.3120e-01,  9.9998e-01,  5.6922e-03,  2.1696e+00, -1.4468e+00,
         1.4330e+01,  3.4577e+00,  3.5565e+00,  3.0000e-01])
tensor([[4.0629],
        [4.0562],
        [4.0342]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 1.8948e-01,  9.9998e-01,  5.6922e-03,  5.5135e-01, -1.6758e+00,
         1.8546e+01,  5.4566e+00,  2.2242e+00,  3.0000e-01])
tensor([[4.1495],
        [4.1627],
        [4.1651]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.5909e-01,  9.9998e-01,  5.6922e-03, -5.7362e+00, -1.8505e+00,
        -3.8480e+00,  7.3132e+00, -4.4068e+00, -3.0000e-01])
tensor([[4.0429],
        [4.0320],
        [4.0049]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([1.3199e-01, 9.9998e-01, 5.6922e-03, 4.1403e+00, 3.0220e-01, 1.3402e+01,
        3.5442e+00, 3.8566e+00, 3.0000e-01])
tensor([[4.1573],
        [4.1686],
        [4.1693]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 1.0431e-01,  9.9998e-01,  5.6922e-03, -5.1671e+00,  

tensor([ 2.4579e-01,  9.9998e-01,  5.6922e-03,  7.0394e-01, -1.0978e+00,
        -1.3139e+01, -2.1084e+00, -1.4991e+00, -3.0000e-01])
tensor([[4.0756],
        [4.0716],
        [4.0529]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 0.2611,  1.0000,  0.0057,  4.2928, -1.3873,  2.2720, -0.7883,  3.3535,
         0.3000])
tensor([[4.0535],
        [4.0448],
        [4.0204]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 2.4831e-01,  9.9998e-01,  5.6922e-03,  1.7467e+00, -1.0153e+00,
         1.1295e+01, -4.2892e-01,  2.9146e+00,  3.0000e-01])
tensor([[4.0447],
        [4.0342],
        [4.0076]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 2.2122e-01,  9.9998e-01,  5.6922e-03,  3.4595e-02, -1.3488e+00,
         1.3000e+01,  1.6602e+00,  1.1514e+00,  3.0000e-01])
tensor([[4.0792],
        [4.0759],
        [4.0581]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 1.8337e-01,  9.9998e-01,  5.6922e-03,  1.8929e+00, -2.4364e+00,
         1.5770

tensor([ 0.1833,  1.0000,  0.0057,  5.6311,  1.6752,  2.9935, -2.6892,  4.8106,
         0.3000])
tensor([[4.1071],
        [4.1095],
        [4.0989]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 0.1881,  1.0000,  0.0057,  0.1254,  1.3615,  0.1595, -3.0467, -0.0631,
        -0.0000])
tensor([[4.1579],
        [4.1683],
        [4.1684]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 2.0611e-01,  9.9998e-01,  5.6922e-03, -2.1609e+00,  1.0783e+00,
        -1.0202e+01, -3.3774e+00, -2.5223e+00, -3.0000e-01])
tensor([[4.1021],
        [4.1035],
        [4.0916]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 0.2235,  1.0000,  0.0057,  1.5131,  0.3565, -0.6266, -2.2944,  1.9108,
        -0.0000])
tensor([[4.1272],
        [4.1337],
        [4.1282]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 2.3199e-01,  9.9998e-01,  5.6922e-03,  1.4441e-01,  1.3478e-02,
        -7.8699e-04, -1.8083e+00,  3.6861e-01, -0.0000e+00])
tensor([[4.1155],
        

...

tensor([ 4.0322e-02,  1.3464e-02, -9.9991e-01,  8.4195e-01, -4.7439e-01,
        -1.6085e+01, -8.0175e-01,  2.9801e+00, -3.0000e-01])
tensor([[4.1572],
        [4.1685],
        [4.1693]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 6.5629e-02,  1.3464e-02, -9.9991e-01,  3.3950e-01, -5.7057e-01,
        -1.4434e+01, -3.8468e-01,  2.5432e+00, -3.0000e-01])
tensor([[4.1569],
        [4.1686],
        [4.1697]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([  0.0942,   0.0135,  -0.9999,  -0.0515,  -0.4863, -13.1061,  -0.2398,
          1.9477,  -0.3000])
tensor([[4.1464],
        [4.1570],
        [4.1564]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([ 0.1088,  0.0135, -0.9999, -0.3641, -2.0279, -0.8972,  2.7527,  0.9676,
        -0.0000])
tensor([[4.1574],
        [4.1685],
        [4.1690]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 0.1219,  0.0135, -0.9999, -0.5139,  2.0335, -8.4052, -1.6241,  0.4013,
        -0.3000])
tensor([[4.1556],

tensor([ 4.0243e-01,  1.3464e-02, -9.9991e-01,  1.0009e+00,  1.0623e+00,
         1.6785e+01,  2.6152e-01,  3.8070e+00,  3.0000e-01])
tensor([[4.1498],
        [4.1611],
        [4.1613]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 0.3751,  0.0135, -0.9999,  0.1676,  4.5911, -3.1232, -4.5346,  2.6177,
        -0.3000])
tensor([[4.1570],
        [4.1686],
        [4.1696]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.8380e-01,  1.3464e-02, -9.9991e-01,  1.2366e-01,  1.4927e+00,
        -1.3673e+01, -3.3997e+00,  2.9012e+00, -3.0000e-01])
tensor([[3.7753],
        [3.8709],
        [3.9480]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 4.0639e-01,  1.3464e-02, -9.9991e-01, -2.4510e-01, -6.5027e-01,
        -1.3713e+01, -8.2222e-01,  1.7596e+00, -3.0000e-01])
tensor([[3.8763],
        [3.8309],
        [3.7614]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 0.4155,  0.0135, -0.9999, -0.4286, -3.6348,  0.5235,  3.2140,  0.8536,
        

tensor([ 4.3857e-01,  1.3464e-02, -9.9991e-01, -4.5646e-01, -5.7523e-01,
        -1.3628e+01, -1.0084e+00,  6.3225e-01, -3.0000e-01])
tensor([[3.7007],
        [3.6189],
        [3.5045]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 0.4641,  0.0135, -0.9999, -0.8172, -4.0077,  2.4827,  3.5417, -0.6519,
         0.3000])
tensor([[3.8082],
        [3.7487],
        [3.6617]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 4.5624e-01,  1.3464e-02, -9.9991e-01, -6.6989e-01, -1.8220e+00,
         1.3526e+01,  3.4081e+00, -7.4900e-01,  3.0000e-01])
tensor([[3.8065],
        [3.7467],
        [3.6593]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 4.3033e-01,  1.3464e-02, -9.9991e-01, -8.8681e-01,  9.3888e-02,
         1.5690e+01,  1.4569e+00, -2.1436e+00,  3.0000e-01])
tensor([[3.9403],
        [3.9082],
        [3.8549]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 3.9295e-01,  1.3464e-02, -9.9991e-01, -1.0169e+00,  1.0029e-01,
         1.7791

tensor([ 0.3193,  0.0135, -0.9999,  1.5285, -0.8646, -0.3283,  1.1123,  3.7295,
        -0.0000])
tensor([[4.1569],
        [4.1687],
        [4.1697]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([  0.3292,   0.0135,  -0.9999,   0.6095,   2.3596, -11.5606,  -2.9025,
          3.7344,  -0.3000])
tensor([[4.1568],
        [4.1686],
        [4.1698]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 3.6219e-01,  1.3464e-02, -9.9991e-01, -6.1499e-01, -2.2889e-01,
        -1.5085e+01, -1.7866e+00,  3.4119e+00, -3.0000e-01])
tensor([[3.9475],
        [3.9168],
        [3.8654]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([ 0.3806,  0.0135, -0.9999, -1.4478, -4.1490,  2.4268,  3.7759,  2.3419,
         0.3000])
tensor([[4.1573],
        [4.1685],
        [4.1693]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ 0.3926,  0.0135, -0.9999, -2.0218,  1.9683, -6.8183, -1.8215,  2.1871,
        -0.3000])
tensor([[4.1536],
        [4.1657],
        [4.1669]]

tensor([ -0.6659,  -0.9478,  -0.3189,   1.3438,  -1.0430, -19.3354,   0.9351,
          3.8048,  -0.3000])
tensor([[2.7069],
        [3.0064],
        [3.2690]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ -0.8379,  -0.9478,  -0.3189,   0.7685,  -0.9640, -17.1936,   1.2643,
          3.5797,  -0.3000])
tensor([[0.1147],
        [0.8521],
        [1.5115]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ -0.9837,  -0.9478,  -0.3189,   0.2174,  -0.6517, -15.6027,   1.1214,
          2.8287,  -0.3000])
tensor([[-1.2986],
        [-0.5760],
        [ 0.1831]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ -1.1091,  -0.9478,  -0.3189,  -0.1306,  -0.3976, -14.2684,   0.9832,
          2.2481,  -0.3000])
tensor([[-4.5053],
        [-3.9036],
        [-3.0535]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([ -1.2189,  -0.9478,  -0.3189,  -0.3448,  -0.1425, -13.1119,   0.6824,
          1.6163,  -0.3000])
tensor([[-6.3300],
        [-5.7214],
        [